In [1]:
import os

In [2]:
%pwd

'd:\\ml projects\\End-To-End-MLOPs-Credit_Card_Fraud_Detection-main\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ml projects\\End-To-End-MLOPs-Credit_Card_Fraud_Detection-main'

Entity Configuration

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators : int
    criterion: str
    bootstrap: bool
    target_column: str

Configuration Managar


In [6]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema =  self.schema.TARGET_COLUMN
        param = self.params.RandomForestClassifier

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = param.n_estimators,
            criterion = param.criterion,
            bootstrap = param.bootstrap,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from Credit_Card_Fraud_Detection import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [9]:
class ModelTrainerStrategy:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        rfc = RandomForestClassifier(n_estimators=self.config.n_estimators, criterion = self.config.criterion,bootstrap = self.config.bootstrap, random_state=42,verbose = True)
        rfc.fit(train_x, train_y)

        joblib.dump(rfc, os.path.join(self.config.root_dir, self.config.model_name))



In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainerStrategy(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-27 23:32:09,092: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-27 23:32:09,098: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-27 23:32:09,102: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-27 23:32:09,102: INFO: common: created directory at: artifacts]
[2024-12-27 23:32:09,102: INFO: common: created directory at: artifacts/model_trainer]


d:\ml projects\End-To-End-MLOPs-Credit_Card_Fraud_Detection-main\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.7min
